In [20]:
import pickle
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.vectorstores import FAISS

In [4]:
index_file_path = '../indexes/langchain_faiss_index_scripts_432.pickle'

#  You can get the pickle files also from - https://drive.google.com/drive/folders/1rWUQbW4gyOvU96Uo8vFSbJH4VTNjAg7g?usp=share_link
with open(index_file_path, 'rb') as handle:
    db = pickle.load(handle)

In [10]:
# To use below command, you need to have the OPENAI_API_KEY environment variable set with your openai key
query = "bianca has relationship with whom in 10 things i hate about you?"
sim_docs = db.similarity_search(query)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI.


In [11]:
len(sim_docs)

4

In [25]:
print(sim_docs[2].page_content)

10 Things I Hate About You script by Karen McCullah Lutz, Kirsten Smith & William Shakespeare















                               TEN THINGS I HATE ABOUT YOU
          
                written by Karen McCullah Lutz & Kirsten Smith
          
              based on 'Taming of the Shrew" by William Shakespeare
          
          Revision November 12, 1997
          
          
          PADUA HIGH SCHOOL - DAY
          
          Welcome to Padua High School,, your typical urban-suburban 
          high school in Portland, Oregon.

Smarties, Skids, Preppies, 
          Granolas.

Loners, Lovers, the In and the Out Crowd rub sleep 
          out of their eyes and head for the main building.

PADUA HIGH PARKING LOT - DAY
          
          KAT STRATFORD, eighteen, pretty -- but trying hard not to be 
          -- in a baggy granny dress and glasses, balances a cup of 
          coffee and a backpack as she climbs out of her battered, 
          baby blue '75 Dodge Dart.

A 

As we can see above, the similarity search function returns the relevent documents but we still need llm to answer the question we asked based on the context of returned relevent docs. To do this we can expose the index to retriever interface in langchain for the vectorstore.

In [18]:
# I have specified k as 3 in below retriever to limit the number of relevent documents to be passed to LLM to 3 to reduce the input tokens. 
# This is done because gpt3 endpoint has a limit of approx 4000 tokens including the response.
retriever = db.as_retriever(search_kwargs={"k": 3}) 
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever)

In [19]:
qa.run(query)

' Bianca has a flirtatious relationship with Joey, but it is not a romantic relationship. She also develops romantic feelings for Cameron, and later starts a relationship with him.'

In my run the result returned was- 

*Bianca has a flirtatious relationship with Joey, but it is not a romantic relationship. She also develops romantic feelings for Cameron, and later starts a relationship with him.*

We can see that LLM was able to answer the question we asked correctly.